In [136]:
from typing import Annotated, Literal,Any
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,END,START
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
from langgraph.types import Command
from langgraph.graph.message import add_messages
from langchain.pydantic_v1 import Field
from langchain.llms import HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI

In [137]:
from dotenv import load_dotenv
import os

In [138]:
load_dotenv()

True

In [139]:
GROQ_API_KEY= os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
print(GROQ_API_KEY)
os.environ["GROQ_API_KEY"] = "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"
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY 
os.environ["SERPER_API_KEY"] = SERPER_API_KEY
os.environ["LANGCHAIN_API_KEY"] =  LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

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


In [140]:
architect_model = ChatGroq(model='deepseek-r1-distill-llama-70b')
optimizer_model = ChatGroq(model='meta-llama/llama-4-maverick-17b-128e-instruct')
critic_model = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct")
requirements_model = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")
arbiter_model = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")


In [141]:
critic_model.invoke("how many days are there in a week")

AIMessage(content='There are 7 days in a week.\n\nThe 7 days of the week are:\n\n1. Monday\n2. Tuesday\n3. Wednesday\n4. Thursday\n5. Friday\n6. Saturday\n7. Sunday', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 18, 'total_tokens': 64, 'completion_time': 0.19123909, 'prompt_time': 0.005283489, 'queue_time': 0.213845468, 'total_time': 0.196522579}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_fe2a81a060', 'finish_reason': 'stop', 'logprobs': None}, id='run--d91943de-807b-481a-8234-1be893dd6e24-0', usage_metadata={'input_tokens': 18, 'output_tokens': 46, 'total_tokens': 64})

In [142]:
architect_model.invoke("how many days are in a week")

AIMessage(content="<think>\nFirst, I need to determine the number of days in a week.\n\nA standard week is commonly understood to have seven days.\n\nThese seven days are typically named as follows: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, and Sunday.\n\nTherefore, the total number of days in a week is seven.\n</think>\n\n**Solution:**\n\nTo determine how many days are in a week, let's break it down step by step.\n\n1. **Understanding the Week:**\n   - A week is a unit of time commonly used to divide the days of the year into manageable periods.\n   - It is based on a cycle of seven days.\n\n2. **Listing the Days:**\n   - The seven days of the week are:\n     1. **Monday**\n     2. **Tuesday**\n     3. **Wednesday**\n     4. **Thursday**\n     5. **Friday**\n     6. **Saturday**\n     7. **Sunday**\n\n3. **Counting the Days:**\n   - By listing them out, we can see that there are **7 days** in total.\n\n**Final Answer:**\n\\[\n\\boxed{7}\n\\]", additional_kwargs={}, respo

In [143]:
class AgentState(TypedDict):
  requirements : str
  messages : Annotated[list[Any],add_messages]
  next : Literal["requirements_node","architect_node","critic_node","optimizer_node","arbiter_node","__end__"] 
  architecture : str
  criticism : str
  optimizer_visits : int
  optimization_details : str

In [144]:
class CriticGrade(TypedDict):
  critic_score : Literal["yes","no"] = Field(description="Used to criticise a software architecture."
  " 'yes' if it needs optimization/improvement, 'no' if the current architecture is optimum ")
  criticism : str = Field(description="Describe the parts where the software architecture needs improvement")

In [145]:
class SadAgent:
  def __init__(self):
    self.architect_model = architect_model
    self.optimizer_model = optimizer_model
    self.arbiter_model = arbiter_model
    self.requirements_model = requirements_model
    self.critic_model = critic_model

  def supervisor_node(self,state : AgentState) -> Command[Literal["requirements_node","architect_node","critic_node","optimizer_node","arbiter_node","__end__"]]:
    print("****************Entered supervisor****************")
    
    
    if state['optimizer_visits'] == 3:
        
        state["messages"].append({
            "role": "system",
            "content": "Exceeded maximum optimizer visits. Proceeding to Arbiter."
        })
        optimization_details = "Skipped Optimatization due to excessive optimization cycles."
        goto = "arbiter_node"

        return Command(update={"optimization_details":optimization_details,"next":goto},goto=goto)
    
    else:
      if len(state['messages']) == 1:
        query = state['messages'][0].content
        print(query)
        state['messages'].append({
          "role":"system",
          "content":"Transitioning to requirements engineer to generate the system requirements based on the User's input"
          })
        
        goto = 'requirements_node'
        return Command(update={"next":goto},goto=goto)
      else:
         goto=state['next']
         state['messages'].append({
            "role":"system",
            "content":f"Transitioning to {state["next"]} to proceed further"
         })
         return Command(goto=goto)

  def requirements_node(self,state : AgentState) -> Command[Literal["supervisor"]]:
     print("****************Entered requirements node****************")
     #requirements_agent = create_react_agent(self.llm_model,tools=[], prompt="You are a requirements engineer. You are supposed to break down the user's software idea and identify the system requirements. Give accurate, appropriate system requirements")
     
     requirements_prompt = "You are a requirements engineer. You are supposed to break down the user's software idea and identify the system requirements. Give accurate, appropriate system requirements"
     #result = requirements_agent.invoke({"input":state['messages'][0].content})

     result = self.requirements_model.invoke(f"{requirements_prompt}\n\n{state['messages'][0].content}")

     return Command(
        update={"messages":state['messages'] + [HumanMessage(content="[REQUIREMENTS ENGINEER] System Requirements has been generated by requirements engineer, Transitioning to Architect",name='requirements_engineer')]
                , "requirements":result.content,"next":"architect_node"},goto="supervisor"
     )
  

  def architect_node(self, state : AgentState) -> Command[Literal["supervisor"]]:
     print("****************Entered architect node****************")
     #architect_agent = create_react_agent(self.llm_model,tools=[],prompt="Based on the system requirements, generate a complete,neat, scalable software architecture design along with appropriate frameworks, libraries, deployment options")
     
     architect_prompt = "Based on the system requirements, generate a complete,neat, scalable software architecture design along with appropriate frameworks, libraries, deployment options"

     result = self.architect_model.invoke(f"{architect_prompt}\n\n{state['requirements']}")

     #result = architect_agent.invoke({"input":f"System Requirements : {state['requirements']}"}) 
     print(result)
     return Command(
        update={"messages": state['messages'] + [HumanMessage(content="[ARCHITECT] Initial Design has been generated by architect, Transitioning to Critic for further evaluation",name="architect")],
                "architecture":result.content,"next":"critic_node"},goto="supervisor"
     )

  def critic_node(self,state : AgentState) -> Command[Literal["supervisor","arbiter_node"]]:
     print("****************Entered critic node****************")
     #critic_agent = create_react_agent(self.llm_model.with_structured_output(CriticGrade),tools=[],prompt="You are a software architecture critic in a multi-agent system.Your job is to carefully examine a proposed system architecture and identify any flaws, anti-patterns, or weaknesses. These may include violations of software " \
     #"design principles, overengineering, security concerns, lack of scalability, or poor modularity.If you find the architecture to be optimal and meets the system requirements, then respond")

     critic_prompt = ("You are a software architecture critic in a multi-agent system.Your job is to carefully examine a proposed system architecture and identify any flaws, anti-patterns, or weaknesses. These may include violations of software " \
     "design principles, overengineering, security concerns, lack of scalability, or poor modularity.If you find the architecture to be optimal and meets the system requirements, then respond")

     result= self.critic_model.with_structured_output(CriticGrade).invoke(f"{critic_prompt} \n\n software architecture: {state['architecture']}")
     #result = critic_agent.invoke({"input":f"Software architecture: {state['architecture']}"})
     
     if result['critic_score'] == 'yes':
        goto = "optimizer_node"
        visits = state['optimizer_visits']+1
        print(visits)
        msg = [HumanMessage(content="[CRITIC] Criticism has been generated. Transitioning to Optimizer",name="critic")]

        return Command(
        update={"messages":state['messages'] + msg,"next":goto,'criticism':result['criticism'],'optimizer_visits':visits},goto="supervisor"
     )

     else:
        goto = 'arbiter_node'
        msg = [HumanMessage(content="[CRITIC] Criticism has been generated. Transitioning to Arbiter",name="critic")]
        return Command(
        update={"messages":state['messages'] + msg,"next":goto,'criticism':result['criticism']},goto="arbiter_node"
     )
  
     
  
  def optimizer_node(self,state : AgentState) -> Command[Literal["supervisor","arbiter_node"]]:
     print("****************Entered optimizer node****************")
     print(f"Optimizer Visits: {state['optimizer_visits']}")
     if state['optimizer_visits'] == 1:
        return Command(goto="arbiter_node")
     #optimizer_agent = create_react_agent(self.llm_model,tools=[],prompt="You are a software architecture optimizer based on the criticism mentioned. Optimize the given architecture keeping the requirements in mind as well.Give only the final software architecture design after optimization")
     
     optimizer_prompt = "You are a software architecture optimizer based on the criticism mentioned. Optimize the given architecture keeping the requirements in mind as well.Give only the final software architecture design after optimization"
     
     #result = optimizer_agent.invoke({"input":f"Software Architecture: {state['architecture']}\n\n Criticism: {state['criticism']} \n\n Requirements: {state['requirements']}"})

     result = self.optimizer_model.invoke(f"{optimizer_prompt}\n\n Software Architecture: {state['architecture']}\n\n Requirements: {state['requirements']}\n\n Criticism: {state['criticism']}")
     print(result)
     return Command(
        update={"messages":state['messages']+[HumanMessage(content="[OPTIMIZER] The architecture has been optimized by the optimzer. Transitioning to the Critic",name="optimizer")],"next":"critic_node","architecture":result},goto="supervisor"
     )
  
  
  def arbiter_node(self, state: AgentState) -> Command[Literal["__end__"]]:
    print("****************Entered arbiter node****************")
    #arbiter_agent = create_react_agent(self.llm_model,tools=[], prompt=(
     #   "You are the Arbiter in a multi-agent architecture design system. "
     #   "Your role is to verify whether the proposed software architecture aligns with the user's business requirements. "
     #  "Carefully assess if the architecture supports the intended goals, use cases, and constraints. "
     #   "If the architecture is fine, then don't change it. If not, modify it accordingly."
    #))

    #result = arbiter_agent.invoke({"input":f"Software Architecture: {state['architecture']}\n\n Requirements: {state['requirements']}"})
    
    sys_prompt =         ("You are the Arbiter in a multi-agent architecture design system. "
        "Your role is to verify whether the proposed software architecture aligns with the user's business requirements. "
        "Carefully assess if the architecture supports the intended goals, use cases, and constraints. "
        "If the architecture is fine, then don't change it. If not, modify it accordingly."
        "Give only the final, complete software architecture design")

    result = self.arbiter_model.invoke(f"{sys_prompt}\n\nSoftware Architecture: {state['architecture']}\n\n Requirements: {state['requirements']}")

    msg = [HumanMessage(content="[ARBITER] Final decision made. Sending architecture to user.", name="arbiter")]

    return Command(
        update={"architecture": result.content, "messages": state["messages"] + msg,"next":"FINISH"},
        goto="__end__"
    )

  
  def workflow(self):
     self.graph = StateGraph(AgentState)
     self.graph.add_node("architect_node",self.architect_node)
     self.graph.add_node("arbiter_node",self.arbiter_node)
     self.graph.add_node("optimizer_node",self.optimizer_node)
     self.graph.add_node("critic_node",self.critic_node)
     self.graph.add_node("requirements_node",self.requirements_node)
     self.graph.add_node("supervisor",self.supervisor_node)
     self.graph.add_edge(START,"supervisor")

     self.app = self.graph.compile()
     return self.app


In [146]:
input = {
    "requirements":"",
    "messages" : [HumanMessage(content="Build an Leave management system for college students. make sure it is scalable and budget friendly")],
  "next" : "requirements_node",
  "architecture" : "",
  "criticism" : "",
  "optimizer_visits" : 0,
  "optimization_details" :""
}

In [147]:
agent = SadAgent()

In [148]:
app_graph = agent.workflow()

In [149]:
app_graph.invoke(input)

****************Entered supervisor****************
Build an Leave management system for college students. make sure it is scalable and budget friendly
****************Entered requirements node****************
****************Entered supervisor****************
****************Entered architect node****************
content="<think>\nOkay, so I'm trying to design a software architecture for a Leave Management System for college students based on the requirements provided. Let me break this down step by step.\n\nFirst, I need to understand the functional requirements. The system needs to handle user registration, leave applications, approval processes, a calendar for leaves, reporting, and notifications. Non-functional requirements include scalability, budget-friendliness, security, usability, and performance. The system should be web-based, responsive, and integrate with existing SIS via APIs. It also needs a solid database schema, data backup, and recovery processes. Hardware-wise, it sh

{'requirements': "As a requirements engineer, I will break down the user's software idea and identify the system requirements for a Leave Management System for college students. Here are the requirements:\n\n**Functional Requirements:**\n\n1. **User Registration and Profile Management:**\n\t* The system should allow college students to register and create a profile with their personal details (name, email, phone number, student ID, etc.).\n\t* Students should be able to edit their profile information.\n2. **Leave Application:**\n\t* Students should be able to apply for leaves by selecting the type of leave (e.g., annual, medical, etc.), specifying the leave dates, and providing a reason for the leave.\n\t* The system should allow students to attach supporting documents (e.g., medical certificates, etc.) for their leave application.\n3. **Leave Approval Process:**\n\t* The system should enable faculty advisors or department heads to view and approve/reject leave applications.\n\t* Appro

In [158]:
import subprocess

# Save UML code to file
uml_code = """
@startuml
Alice -> Bob: Hello
Bob --> Alice: Hi!
@enduml
"""

with open("example.puml", "w") as f:
    f.write(uml_code)

jar_path = "./plantuml-1.2025.2.jar"
java_path = "C:/Program Files/Java/jdk-21/bin/java.exe"
# Run PlantUML JAR to generate the diagram
# Make sure plantuml.jar is in the same directory or give full path
subprocess.run([java_path, "-jar", jar_path, "C:/Users/thede/OneDrive/Desktop/ete_backend/example.puml"], check=True)

print("✅ Diagram generated: example.png")


✅ Diagram generated: example.png
